In [2]:
training_folder = 'training_data/'

In [3]:
import numpy as np

# instantiate empty arrays for features and labels
Xtr = np.array([])
ytr = np.array([])
k = 0 # initialize

# load data from the relevant files
while True:
    try:
        # load data file
        class_k = np.loadtxt(training_folder + 'Class{:}.csv'.format(k))
        # extract features and labels
        class_k_features = class_k[:,:-1] # extract features
        class_k_labels  = class_k[:,-1].astype(np.int) # labels; convert to int
        # append the features and labels to the arrays
        Xtr = np.vstack([Xtr,class_k_features]) if Xtr.size else class_k_features
        ytr = np.hstack([ytr,class_k_labels]) if ytr.size else class_k_labels
        # increment counter
        k += 1
    except:
        print('loaded %i classes of training data' %k)
        break

# examine shape
num_classes = k
num_features = Xtr.shape[1]
num_samples = Xtr.shape[0]

print('unique labels: ', np.unique(ytr))
print('number of features: ', num_features)
print('number of samples: ', num_samples)

loaded 20 classes of training data
unique labels:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
number of features:  20
number of samples:  100000


In [4]:
#Splitting the dataset into training and testing 

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xtr, ytr, test_size = 0.33, random_state = 0)

In [ ]:
pip install xgboost

In [8]:
#Fitting XGBoost to the training set 

from xgboost import XGBClassifier
classifier = XGBClassifier(max_depth=4)
classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [11]:
## Predicting the Test set results
y_pred_train = classifier.predict(X_train)
y_pred = classifier.predict(X_test)

In [12]:
acc_train = np.mean(y_pred_train == y_train)
acc = np.mean(y_pred == y_test)
print('Accuracy on train data = {0:.4f}'.format(acc_train))
print('Accuracy on test data = {0:.4f}'.format(acc))

Accuracy on train data = 0.9609
Accuracy on test data = 0.8973


In [13]:
# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
accuracies.mean()
accuracies.std()

0.003050343357459284

In [14]:
print(accuracies)
accuracies.mean

[0.89399135 0.89278258 0.89813572 0.89349642 0.8980597  0.89207344
 0.89371548 0.90128435 0.89556253 0.89986549]
